In [20]:
import time
import pandas as pd
from review_scraper import AmazonScraper
import os

In [19]:
#questa cella scrapa le recensioni

amz_scraper = AmazonScraper()

products_url = {"echo_dot.csv":"https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R/ref=cm_cr_arp_d_viewopt_rvwer?ie=UTF8&reviewerType=avp_only_reviews&sortBy=recent&pageNumber=1",
                "echo_4thgen.csv":"https://www.amazon.com/Echo-4th-Gen/product-reviews/B07XKF5RM3/ref=cm_cr_arp_d_viewopt_rvwer?ie=UTF8&reviewerType=avp_only_reviews&sortBy=recent&pageNumber=1",
                "echo_show10.csv":"https://www.amazon.com/echo-show-10/product-reviews/B07VHZ41L8/ref=cm_cr_arp_d_viewopt_rvwer?ie=UTF8&reviewerType=avp_only_reviews&sortBy=recent&pageNumber=1"}

#se hai bisogno di capire come trovare la url giusta scrivimi

for product in products_url:
    reviews = []
    for page in range(1, 50):
        response = amz_scraper.scrapeReviews(url=products_url[product], page_num=page, filter_by="recent")
        reviews.extend(response)
        time.sleep(1)

    df = pd.DataFrame(reviews)

    #######################################################
    #pulisce i dati

    ratings = []
    feedbacks = []

    for score in df["rating"]:
        try:
            value = int(score[0])
        except TypeError:
            print("Errore sul valore:", score)
        if value <= 2:
            feedbacks.append(0)
        elif value >= 3:
            feedbacks.append(1)

        ratings.append(value)

    df["rating"] = ratings
    df["feedback"] = feedbacks
    df.drop(["review_title", "username", "profile_url", "verified_purchase"], axis=1, inplace=True)
    df.rename({"product_name":"variation", "comment":"verified_reviews"}, inplace=True, axis=1)

    #######################################################

    df.to_csv(f"scraped_data/{product}", sep="\t", index = False) #salva


Processing https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R?reviewerType=all_reviews&sortBy=recent&pageNumber=1...
Processing https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R?reviewerType=all_reviews&sortBy=recent&pageNumber=2...
Processing https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R?reviewerType=all_reviews&sortBy=recent&pageNumber=3...
Processing https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R?reviewerType=all_reviews&sortBy=recent&pageNumber=4...
Processing https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R?reviewerType=all_reviews&sortBy=recent&pageNumber=5...
Processing https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R?reviewerType=all_reviews&sortBy=recent&pageNumber=6...
Processing https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R?reviewerType=all_reviews&sortBy=recent&pageNumber=7...
Processing https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R?reviewerType=all_reviews&sortBy=recent&pageNumber=8...


In [29]:
#################################
#unisce tutti i df estratti in uno unico
df_list = []

for path in os.listdir("scraped_data"):
    df = pd.read_csv(f"scraped_data/{path}", sep = "\t")
    df_list.append(df)

total_df = pd.concat(df_list)

total_df

,variation,verified_reviews,rating,date,feedback
0,Echo 4th Gen,I wish we never purchased this I hate the alex...,1,"October 25, 2022",0
1,Echo 4th Gen,"Cons: it cannot Cast audio from my phone, does...",2,"October 25, 2022",0
2,Echo 4th Gen,Love this Echo. It is bigger than expected but...,4,"October 25, 2022",1
3,Echo 4th Gen,First off I would give this premium sound echo...,4,"October 25, 2022",1
4,Echo 4th Gen,Hubiera dado mejor puntuación si hubiera podid...,4,"October 24, 2022",1
...,...,...,...,...,...
485,echo show 10,Product seems fine. Dislike the unexpected ads...,4,"August 4, 2022",1
486,echo show 10,I really like it.,5,"August 4, 2022",1
487,echo show 10,Roomie got this 'thing' from a friend as a gif...,1,"August 4, 2022",0
488,echo show 10,Excelente producto a la medida. Lo recomiendo ...,5,"August 4, 2022",1


In [30]:
total_df["feedback"].value_counts()

1    1275
0     195
Name: feedback, dtype: int64

In [ ]:
# Ci sono alcune reviews in spagnolo (le recensioni vengono amazon.com, quindi america), si può valutare di tradurle.

In [31]:
total_df.to_csv("test_set.csv", sep = "\t") #salva il df totale come test set